In [ ]:
import json
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.features import CustomIcon
import openrouteservice
from branca.element import Figure
from openrouteservice import convert

# JSONファイルの読み込み
with open('secret.json') as f:
    keys = json.load(f)

API_KEY = keys['key']
client = openrouteservice.Client(key=API_KEY)

# このCSVには   、県庁所在地の緯度・経度がlatitudeカラムとlongitudeカラムに入っている。
df_prefecture = pd.read_csv("pref_lat_lon.csv")

def visualize_locations_with_marker_cluster(df, zoom=4):
    """日本を拡大した地図に、pandasデータフレームのlatitudeおよびlongitudeカラムをプロットする。
    """

    # 図の大きさを指定する。
    f = folium.Figure(width=1000, height=500)

    # 初期表示の中心の座標を指定して地図を作成する。
    center_lat=34.686567
    center_lon=135.52000
    m = folium.Map([center_lat,center_lon], zoom_start=zoom).add_to(f)

    # 縮尺によって、マーカーがまとめられるように設定する。
    marker_cluster = MarkerCluster().add_to(m)

    # icon = CustomIcon(
    # icon_image = 'nico.png'
    # ,icon_size = (55, 65)
    # ,icon_anchor = (30, 30)
    # ,popup_anchor = (3, 3)
    # )

    p1 = 35.689185,139.691648
    p2 = 35.447505,139.642347
    p1r = tuple(reversed(p1))
    p2r = tuple(reversed(p2))

    # 経路計算 (Directions V2)
    routedict = client.directions((p1r, p2r))
    geom = routedict["routes"][0]["geometry"]
    decoded = convert.decode_polyline(geom)

    def reverse_lat_long(list_of_lat_long):
        """緯度経度をひっくり返す"""
        return [(p[1], p[0]) for p in list_of_lat_long]

    coord = reverse_lat_long(decoded["coordinates"])

    marker_cluster = folium.vector_layers.PolyLine(locations=coord).add_to(marker_cluster)
        
    # データフレームの全ての行のマーカーを作成する。
    for i in range(0,len(df)):
        folium.Marker(
            location=[df["lat"][i],df["lon"][i]],
            popup=df["pref_name"],
            profile="foot-walking"
            # icon = icon
            ).add_to(marker_cluster)
        
    return m
    
visualize_locations_with_marker_cluster(df_prefecture, zoom=3)

In [ ]:
routedict["routes"][0]['summary']

{'distance': 33936.1, 'duration': 2193.6}

In [ ]:
from pyrosm.data import sources

sources.available.keys()
sources.asia.japan.available